### Imports

In [1]:
import os
import csv
import re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
import cv2

/opt/conda/envs/SpaGCN/lib/python3.8/site-packages/tqdm-4.64.1-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spg.__version__

'1.2.5'

### Read in data
The current version of SpaGCN requres three input data: 
<br>
1. The gene expression matrix(n by k): expression_matrix.h5;
<br>
2. Spatial coordinateds of samplespositions.txt;
<br>
3. Histology image(optional): histology.tif, can be tif or png or jepg.
<br>
The gene expreesion data can be stored as an AnnData object. AnnData stores a data matrix .X together with annotations of observations .obs, variables .var and unstructured annotations .uns. 

In [14]:
# Read original 10x_h5 data and save it to h5ad
from scanpy import read_10x_h5
adata = read_10x_h5("./data/Train_filtered_feature_bc_matrix.h5")
spatial=pd.read_csv("./data/Train_tissue_positions_list.csv",sep=",",header=None,na_filter=False,index_col=0) 
adata.obs["x1"]=spatial[1]
adata.obs["x2"]=spatial[2]
adata.obs["x3"]=spatial[3]
adata.obs["x4"]=spatial[4]
adata.obs["x5"]=spatial[5]
adata.obs["x_array"]=adata.obs["x2"]
adata.obs["y_array"]=adata.obs["x3"]
adata.obs["x_pixel"]=adata.obs["x4"]
adata.obs["y_pixel"]=adata.obs["x5"]
#Select captured samples
adata=adata[adata.obs["x1"]==1]
adata.var_names=[i.upper() for i in list(adata.var_names)]
adata.var["genename"]=adata.var.index.astype("str")
adata.write_h5ad("./data/Train_filtered_feature_bc_matrix.h5ad")

In [15]:
#Read in gene expression and spatial location
adata=sc.read("./data/Train_filtered_feature_bc_matrix.h5ad")

#Read in histology image

# img=cv2.imread("./data/Train_tissue_hires_image.png")
# print(img.shape)

img=cv2.imread("./data/histology.tif")
print(img.shape)

(13332, 13332, 3)


### Integrate gene expression and histology into a Graph

In [16]:
#Set coordinates
x_array=adata.obs["x_array"].tolist()
y_array=adata.obs["y_array"].tolist()
x_pixel=adata.obs["x_pixel"].tolist()
y_pixel=adata.obs["y_pixel"].tolist()

#Test coordinates on the image
img_new=img.copy()
for i in range(len(x_pixel)):
    x=x_pixel[i]
    y=y_pixel[i]
    img_new[int(x-20):int(x+20), int(y-20):int(y+20),:]=0

cv2.imwrite('./data/graph.jpg', img_new)

True

- The "s" parameter determines the weight given to histology when calculating Euclidean distance between every two spots. ‘s = 1’ means that the histology pixel intensity value has the same scale variance as the (x,y) coordinates, whereas higher value of ‘s’ indicates higher scale variance, hence, higher weight to histology, when calculating the Euclidean distance. 

- The "b" parameter determines the area of each spot when extracting color intensity.

In [17]:
#Calculate adjacent matrix
s=1
b=7
adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, x_pixel=x_pixel, y_pixel=y_pixel, image=img, beta=b, alpha=s, histology=True)
#If histlogy image is not available, SpaGCN can calculate the adjacent matrix using the fnction below
#adj=calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)
np.savetxt('./data/adj.csv', adj, delimiter=',')

Calculateing adj matrix using histology image...
Var of c0,c1,c2 =  161.4652256456971 335.8159886942832 247.4194039215306
Var of x,y,z =  5606737.526317932 4468793.817921193 5606737.526317932


### Spatial domain detection using SpaGCN

#### Expression data preprocessing

In [18]:
adata=sc.read("./data/Train_filtered_feature_bc_matrix.h5ad")
adj=np.loadtxt('./data/adj.csv', delimiter=',')
adata.var_names_make_unique()
spg.prefilter_genes(adata,min_cells=3) # avoiding all genes are zeros
spg.prefilter_specialgenes(adata)
#Normalize and take log for UMI
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

#### Set hyper-parameters

- p: Percentage of total expression contributed by neighborhoods
- l: Parameter to control p

In [19]:
p=0.5 
#Find the l value given p
l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)

Run 1: l [0.01, 1000], p [0.0, 126.69566843060348]
Run 2: l [0.01, 500.005], p [0.0, 20.448318434833077]
Run 3: l [0.01, 250.0075], p [0.0, 2.7987032017695967]
Run 4: l [125.00874999999999, 250.0075], p [0.3148686083287422, 2.7987032017695967]
Run 5: l [125.00874999999999, 187.508125], p [0.3148686083287422, 1.1667931307575095]
Run 6: l [125.00874999999999, 156.2584375], p [0.3148686083287422, 0.656616908497677]
Run 7: l [140.63359375, 156.2584375], p [0.46648887913565273, 0.656616908497677]
Run 8: l [140.63359375, 148.446015625], p [0.46648887913565273, 0.5565096307109916]
Run 9: l [140.63359375, 144.5398046875], p [0.46648887913565273, 0.5102668871574176]
Run 10: l [142.58669921875, 144.5398046875], p [0.48807337109100546, 0.5102668871574176]
recommended l =  143.56325195312502


- n_clusters: Number of spatial domains wanted.
- res: Resolution in the initial Louvain's Clustering methods. If the number of clusters is known, we can use the spg.search_res() function to search for suitable resolution(optional)

> NOTE: Since louvain's implementation is causing some bug due to excessive RAM usage, we modified the source code of SpaGCN repository shifted to leidenalg clustering method.

In [20]:
#If the number of clusters known, we can use the spg.search_res() fnction to search for suitable resolution(optional)
#For this toy data, we set the number of clusters=7 since this tissue has 7 layers
n_clusters=7
#Set seed
r_seed=t_seed=n_seed=100
#Seaech for suitable resolution
res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)

Start at res =  0.7 step =  0.1
Initializing cluster centers with leidenalg, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  9
Initializing cluster centers with leidenalg, resolution =  0.6
Epoch  0
Epoch  10
Res =  0.6 Num of clusters =  8
Res changed to 0.6
Initializing cluster centers with leidenalg, resolution =  0.5
Epoch  0
Epoch  10
Res =  0.5 Num of clusters =  5
Step changed to 0.05
Initializing cluster centers with leidenalg, resolution =  0.5499999999999999
Epoch  0
Epoch  10
Res =  0.5499999999999999 Num of clusters =  6
Step changed to 0.025
Initializing cluster centers with leidenalg, resolution =  0.575
Epoch  0
Epoch  10
Res =  0.575 Num of clusters =  7
recommended res =  0.575


In [21]:
res

0.575

#### Run SpaGCN

In [22]:
clf=spg.SpaGCN()
clf.set_l(l)
#Set seed
random.seed(r_seed)
torch.manual_seed(t_seed)
np.random.seed(n_seed)
#Run
clf.train(adata,adj,init_spa=True,init="leidenalg",res=res, tol=5e-3, lr=0.05, max_epochs=200,n_clusters=7)
y_pred, prob=clf.predict()
adata.obs["pred"]= y_pred
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Do cluster refinement(optional)
#shape="hexagon" for Visium data, "square" for ST data.
adj_2d=spg.calculate_adj_matrix(x=x_array,y=y_array, histology=False)
refined_pred=spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
adata.obs["refined_pred"]=refined_pred
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')
#Save results
adata.write_h5ad("./results/results.h5ad")

Initializing cluster centers with leidenalg, resolution =  0.575
Epoch  0
Epoch  10
Epoch  20
Epoch  30
Epoch  40
Epoch  50
Epoch  60
Epoch  70
Epoch  80
Epoch  90
Epoch  100
Epoch  110
Epoch  120
Epoch  130
Epoch  140
Epoch  150
Epoch  160
Epoch  170
Epoch  180
Epoch  190
Calculateing adj matrix using xy only...


#### Plot spatial domains

In [23]:
adata=sc.read("./results/results.h5ad")
#Set colors used
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
#Plot spatial domains
domains="pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
ax=sc.pl.scatter(adata,alpha=1,x="y_pixel",y="x_pixel",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("./results/pred.png", dpi=600)
plt.close()

#Plot refined spatial domains
domains="refined_pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
ax=sc.pl.scatter(adata,alpha=1,x="y_pixel",y="x_pixel",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("./results/refined_pred.png", dpi=600)
plt.close()

#### Output csv

In [27]:
adata.obs["pred"].to_csv("./results/pred_output.csv", header=False)
adata.obs["refined_pred"].to_csv("./results/refined_pred_output.csv", header=False)

## References

- https://github.com/jianhuupenn/SpaGCN
- https://www.nature.com/articles/s41592-021-01255-8